<a href="https://colab.research.google.com/github/Wissance/TaxesCalculator/blob/master/taxes_ru_individual_entrepreneur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# ################################ Описание ###################################
# Проект: TaxesRu
# Версия: v.0.2
# Этот проект мы создали для расчета налогов и взносов, которые, в частности, 
# мне лично нужно уплатить при ведении деятельности в качестве
# Индивидуального Предпринимателя (ИП), при этом сейчас (в этой версии) 
# мы рассматриваем следующие условия:
# 1. Отсуствуют наемные работники
# 2. Система налогооблажения - УСН 6% (в будщем будет УСН 15%)
# 3. Срок начала действия патента может несовпадать с графиком начала платежей,
# т.е. часть налогов расчитываются как 6% от доходов
# КОНЕЧНО, довольно много сервисов, которые позволяют делать расчеты, НО я
# НЕ ХОЧУ им платить (Например, банк Точка для учета патента
# требует использование полного пакета услуг, а это около 5000 руб в год,
# поэтому имеет смысл сэкономить). Я НЕ ПРЕТЕНДУЮ на РОЛЬ СПАСИТЕЛЯ ЧЕЛОВЕЧЕСТВА
# и НЕ ПЛАНИРУЮ заменить инструменты такие как Контур, TaxCom, "Мое дело" и др.
# Я лишь хочу СЭКОНОМИТь немного деньжат, и буду рад если мой калькулятор
# кому-то поможет рассчитать его налоги и взносы
# АВТОР проекта (Ушаков Михаил, https://github.com/EvilLord666) не несет НИКАКОЙ
# ГАРАНТИИ, вы используете данный калькулятор на свой СТРАХ и РИСК.
# ОДНАКО я БУДУ ВАМ ОЧЕНЬ ПРИЗНАТЕЛЕН если вы поддержите МОЮ РАБОТУ, например,
# одним из следующих способов:
#     1. Поставить звездочку проекту на Github (https://github.com/Wissance/TaxesCalculator)
#     2. Если вам интересно развитие этого проекта, то заведите задачу на
#        github (https://github.com/Wissance/TaxesCalculator) 
#        и напишите что бы вам хотелось, чтобы я добавил в плане
#        удобств расчета / новый механизм расчета / какие-то аспекты, которые
#        помогут автоматизировать аспекты вашей деятельности

import typing
from datetime import date, datetime
from calendar import monthrange
from abc import ABCMeta, abstractmethod
from functools import reduce

# Обязательные Взносы в ФОМС (Фонд обязательного медицинского страхования)
required_medical_insurance = {
    # формат: год, сумма взносов в руб
    2019: 6884,
    2020: 8426
}

# Обязательные Взносы в ПФР (ОПС, Обязательное пенсионное страхование)
required_retirement_insurance = {
    # формат: год, сумма взносов в руб
    2019: 29354,
    2020: 32448
}

income_overcome = {
    2019: 300000,
    2020: 300000
}

income_overcome_percent = {
    2019: 0.01,
    2020: 0.01
}

INCOME_CODE = 0
EXPENDITURE_CODE = 1
TAXES_EXPENDITURE_CODE = 2
MEDICAL_INSURANCE_EXPENDITURE_CODE = 3
RETIREMENT_INSURANCE_EXPENDITURE_CODE = 4

today = date.today()

NUM_QUARTERS = 4

quarters = {
    1: { 'start' : date(today.year, 1, 1), 'end': date(today.year, 3, 31) },
    2: { 'start' : date(today.year, 4, 1), 'end': date(today.year, 6, 30) },
    3: { 'start' : date(today.year, 7, 1), 'end': date(today.year, 9, 30) },
    4: { 'start' : date(today.year, 10, 1), 'end': date(today.year, 12, 31) }
}

quarters_due_dates = {
    1: date(today.year, 4, 25),
    2: date(today.year, 7, 25),
    3: date(today.year, 10, 25),
    4: date(today.year, 1, 25)
}

# Класс представляющий собой движение средств на текущем этапе это: сумма и дата
class MoneyMove(metaclass=ABCMeta):
    def __init__(self, move_date: date, value: float, movement_type: int, quarter: typing.Optional[int]):
        self.__move_date = move_date
        self.__value = value
        self.__movement_type = movement_type
        self.__quarter = quarter

    @abstractmethod
    def get_value(self) -> float:
        return self.__value

    def get_raw_value(self) -> float:
        return self.__value

    def get_move_date(self) -> date:
        return self.__move_date

    def get_movement_type(self) -> int:
        return self.__movement_type

    def get_quarter(self) -> typing.Optional[int]:
        return self.__quarter

    # данные класса
    __move_date = None
    __value = 0.0
    __movement_type = 0
    __quarter = None


# Класс представляющий собой доход (приходную операцию)
class Income(MoneyMove):
    def __init__(self, move_date: date, value: float, movement_type: int):
        super(Income, self).__init__(move_date, value, movement_type, None)

    def get_value(self) -> float:
        value = self.get_raw_value()
        if value < 0:
            raise ValueError("Доход не может быть отрицательным")
        return value


# Класс представляющий собой расход (приходную операцию)
class Expenditure(MoneyMove):
    def __init__(self, move_date: date, value: float, movement_type: int, quarter: typing.Optional[int]):
        super(Expenditure, self).__init__(move_date, value, movement_type, quarter)

    # Для расходов представляем value в виде отрицательного числа
    # для удобства суммирования значений
    def get_value(self) -> float:
        value = self.get_raw_value()
        if value < 0:
            return value
        return -1 * value

# ##############################################################################

In [81]:
# ############################ Движок калькулятора #############################
# Код который будет учитывать патент и считать налоги и отчисления на основе
# коллекций объектов MoneyMove, Income и Expenditure
# ##############################################################################


class MoneyStats:
    def __init__(self, is_actual: bool, start_date: date, end_date: date, 
                 incomes: float = 0.0, expenditures: float = 0.0,
                 taxes: float = 0.0, taxes_paid: float = 0.0, 
                 medical_insurance_fees: float = 0.0, medical_insurance_fees_paid: float = 0.0,
                 retirement_insurance_fees: float = 0.0, retirement_insurance_fees_paid: float = 0.0,
                 extra_retirement_insurance_fees: float = 0.0, 
                 extra_retirement_insurance_fees_paid: float = 0.0):
        self.__start_date = start_date
        self.__end_date = end_date
        self.__incomes = incomes
        self.__expenditures = expenditures
        self.__taxes = taxes
        self.__taxes_paid = taxes_paid
        self.__medical_insurance_fees = medical_insurance_fees
        self.__medical_insurance_fees_paid = medical_insurance_fees_paid
        self.__retirement_insurance_fees = retirement_insurance_fees
        self.__retirement_insurance_fees_paid = retirement_insurance_fees_paid
        self.__extra_retirement_insurance_fees = extra_retirement_insurance_fees
        self.__extra_retirement_insurance_fees_paid = extra_retirement_insurance_fees_paid

    def get_start_date(self) -> date:
        return self.__start_date

    def get_end_date(self) -> date:
        return self.__end_date

    def get_incomes(self) -> float:
        return self.__incomes

    def get_expenditures(self) -> float:
        return self.__expenditures
    
    def get_taxes(self) -> float:
        return self.__taxes

    def set_taxes(self, taxes: float):
        self.__taxes = taxes

    def get_taxes_paid(self) -> float:
        return self.__taxes_paid

    def get_medical_insurance_fees(self) -> float:
        return self.__medical_insurance_fees

    def get_medical_insurance_fees_paid(self) -> float:
        return self.__medical_insurance_fees_paid

    def get_retirement_insurance_fees(self) -> float:
        return self.__retirement_insurance_fees

    def get_retirement_insurance_fees_paid(self) -> float:
        return self.__retirement_insurance_fees_paid

    def get_extra_retirement_insurance_fees(self) -> float:
        return self.__extra_retirement_insurance_fees

    def get_extra_retirement_insurance_fees_paid(self) -> float:
        return self.__extra_retirement_insurance_fees_paid
    
    def get_total_retirement_insurance_fees(self) -> float:
        return self.__retirement_insurance_fees + self.__extra_retirement_insurance_fees

    def get_total_retirement_insurance_fees_paid(self) -> float:
        return self.__retirement_insurance_fees_paid + self.__extra_retirement_insurance_fees_paid

    __is_actual = True
    __start_date = None
    __end_date = None
    __incomes = 0.0
    __expenditures = 0.0
    __taxes = 0.0  # ФНС
    __taxes_paid = 0.0
    __medical_insurance_fees = 0.0  # ФСС
    __medical_insurance_fees_paid = 0.0
    __retirement_insurance_fees = 0.0  # ПФР
    __retirement_insuarance_fees_paid = 0.0
    __extra_retirement_insurance_fees = 0.0 # 1% Сверх лимита
    __extra_retirement_insuarance_fees_paid = 0.0


class YearMoneyStats:
    def __init__(self, year_stats: MoneyStats, quarters_stats: typing.Dict[int, MoneyStats]):
        self.__year_stats = year_stats
        self.__quarters_stats = quarters_stats

    def get_year_stats(self) -> MoneyStats:
        return self.__year_stats

    def get_quarters_stats(self) -> typing.Dict[int, MoneyStats]:
        return self.__quarters_stats

    #todo: Добавить метод для вывода информации по налогу за год ....

    __year_stats = None
    __quarters_stats = {}

class Calculator(metaclass=ABCMeta):
    @abstractmethod
    def calculate(self, money_moves: typing.List[MoneyMove], year: int, *args, **kwargs) -> YearMoneyStats:
        raise NotImplementedError

    def select(self, money_moves: typing.List[MoneyMove], quarter: int, year: int) -> typing.List[MoneyMove]:
       """
          Выполняем выборку по следующему критерию:
              1. Все доходные операции, которые попадают в период
              2. Все расходные операции с платежами в ФОПС, ФОМС, ФНС до due_date для данного периода
       """
       quarter_item = quarters[quarter]
       begin_date = quarter_item['start'].replace(year=year)
       end_date = quarter_item['end'].replace(year=year)
       if quarter == 4:
           quarter_due_date = quarters_due_dates[quarter].replace(year=year+1)
       else:
           quarter_due_date = quarters_due_dates[quarter].replace(year=year)
       selected_money_moves = list(filter(lambda move: (move.get_move_date() >= begin_date and 
                                                        move.get_move_date() <= end_date and
                                                        move.get_movement_type() <= EXPENDITURE_CODE) or
                                                        (move.get_movement_type() > EXPENDITURE_CODE and move.get_quarter() == quarter), money_moves))
       
       return selected_money_moves

    def get_quarter_insurance_fees(self, begin_date: date, end_date: date, register_date: date, year_fees: float) -> float:
        quarter_month = 3
        full_months = 3
        insurance_fees_coeff = 1.0
        if begin_date.year == register_date.year and begin_date.month <= register_date.month:
            delta = quarter_month - (end_date.month - register_date.month)
            full_months -= delta
            register_month_days = monthrange(register_date.year, register_date.month)[1]
            incomplete_month_days = register_month_days - register_date.day
            insurance_fees_coeff = (full_months + incomplete_month_days / register_month_days) / float(quarter_month)
        fees = (year_fees / 4.0) * insurance_fees_coeff
        # print("Full month: " + str(full_months))  
        # print("Fees coeff: " + str(insurance_fees_coeff))
        # print("Fees : " + str(fees))
        return fees
        

# ############################################################################
# Класс для расчета налогов и сумм взносов для УСН 6%
# ############################################################################
class IncomeTaxCalculator(Calculator):
    def __init__(self, tax_percent: float, register_date: date, 
                 use_patent: bool, patent_start: typing.Optional[date],
                 patent_potential_income: typing.Optional[float]):
        self.__tax_percent = tax_percent
        self.__register_date = register_date
        self.__use_patent = use_patent
        self.__patent_start = patent_start
        self.__patent_potential_income = patent_potential_income

    def calculate(self, money_moves: typing.List[MoneyMove], year: int, *args, **kwargs) -> YearMoneyStats:
        quarters_stats = {}
        for q in range(1, NUM_QUARTERS + 1):
            qua, quarter_stats = self.__get_for_quarter(money_moves, year, q)
            quarters_stats[q] = quarter_stats
        # todo: Реализовать аккумулирующий расчет налогов ....
        incomes_total = 0.0
        expenditures_total = 0.0
        taxes_total = 0.0
        taxes_paid_total = 0.0
        medical_insurance_total = 0.0
        medical_insurance_paid_total = 0.0
        retirement_insurance_total = 0.0
        retirement_insurance_paid_total = 0.0

        for q, s in quarters_stats.items():
            taxes_total += s.get_taxes() - s.get_taxes_paid() - s.get_medical_insurance_fees_paid() - s.get_retirement_insurance_fees_paid()
            # todo: установка, добавить метод ...
            incomes_total += s.get_incomes()
            expenditures_total += s.get_expenditures()
            s.set_taxes(taxes_total)
            taxes_paid_total += s.get_taxes_paid()
            # print("Medical insuarance fees for quarter {0} is: {1}".format(q, s.get_medical_insurance_fees()))
            medical_insurance_total += s.get_medical_insurance_fees()
            medical_insurance_paid_total += s.get_medical_insurance_fees()
            retirement_insurance_total += s.get_retirement_insurance_fees()
            # print("Retirement insuarance fees for quarter {0} is: {1}".format(q, s.get_retirement_insurance_fees()))
            retirement_insurance_paid_total += s.get_total_retirement_insurance_fees_paid()
        year_begin_date = date(year, 1, 1)
        if year_begin_date < self.__register_date:
            year_begin_date = self.__register_date
        year_end_date = date(year, 12, 31)

        tax_base = 0.0
        extra_retirement_insurance_fees = 0.0
        incomes = list(filter(lambda item: type(item) == Income, money_moves))
        for income in incomes:
            if self.__use_patent:
                income_date = income.get_move_date()
                if income_date < self.__patent_start:
                    tax_base += income.get_value()
            else:
                tax_base += income.get_value()
        if self.__use_patent:
            tax_base += self.__patent_potential_income
        if tax_base > income_overcome[year]:
            extra_retirement_insurance_fees = (tax_base - income_overcome[year]) * income_overcome_percent[year]
        #print("Extra retirement insurance payment: " + str(extra_retirement_insurance_fees))

        year_integrated_stats = MoneyStats(True, year_begin_date, year_end_date, incomes_total, expenditures_total, 
                                           taxes_total, taxes_paid_total, 
                                           medical_insurance_total, medical_insurance_paid_total,
                                           retirement_insurance_total, retirement_insurance_paid_total, extra_retirement_insurance_fees)
            
        full_stats = YearMoneyStats(year_integrated_stats, quarters_stats)
        return full_stats

    def __get_for_quarter(self, money_moves: typing.List[MoneyMove], year: int, quarter: int) -> typing.Tuple[int, MoneyStats]:
        quarter_item = quarters[quarter]
        begin_date = quarter_item['start'].replace(year=year)
        end_date = quarter_item['end'].replace(year=year)
        if self.__register_date > end_date:
            return quarter, MoneyStats(False, begin_date, end_date)

        quarter_money_moves = self.select(money_moves, quarter, year)
        incomes = list(filter(lambda item: type(item) == Income, quarter_money_moves))
        expenditures = list(filter(lambda item: type(item) == Expenditure, quarter_money_moves))
        taxes_expenditures = list(filter(lambda item: item.get_movement_type() == TAXES_EXPENDITURE_CODE and
                                                      item.get_quarter() == quarter, expenditures))
        medical_insurance_expenditures = list(filter(lambda item: item.get_movement_type() == MEDICAL_INSURANCE_EXPENDITURE_CODE and
                                                            item.get_quarter() == quarter, expenditures))
        retirement_insurance_expenditures = list(filter(lambda item: item.get_movement_type() == RETIREMENT_INSURANCE_EXPENDITURE_CODE and
                                                               item.get_quarter() == quarter, expenditures))
        # Расчитать сумму дохода с учетом использования патента
        total_incomes = 0.0
        tax_base = 0.0
        total_expenditures = 0.0
        total_taxes_paid = 0.0
        total_medical_insurance_paid = 0.0
        total_retirement_insurance_paid = 0.0

        for income in incomes:
            total_incomes += income.get_value()
            # Проверка даты
            if self.__use_patent:
                income_date = income.get_move_date()
                if income_date < self.__patent_start:
                    tax_base += income.get_value()
            else:
                tax_base += income.get_value()

        # todo: использовать reduce!
        for expenditure in expenditures:
            total_expenditures += expenditure.get_value()

        for taxes_expenditure in taxes_expenditures:
            total_taxes_paid += taxes_expenditure.get_value()

        for medical_insurance_expenditure in medical_insurance_expenditures:
            total_medical_insurance_paid += medical_insurance_expenditure.get_value()

        for retirement_insurance_expenditure in retirement_insurance_expenditures:
            total_retirement_insurance_paid += retirement_insurance_expenditure.get_value()
            
        # print("Total incomes: " + str(total_incomes))
        # print("Tax base: " + str(tax_base))
        taxes = tax_base * self.__tax_percent
        medical_insurance = self.get_quarter_insurance_fees(begin_date, end_date, self.__register_date, 
                                                            required_medical_insurance[year])
        retirement_insurance = self.get_quarter_insurance_fees(begin_date, end_date, self.__register_date, 
                                                               required_retirement_insurance[year])
        return quarter, MoneyStats(True, begin_date, end_date, total_incomes,
                                   total_expenditures, taxes, total_taxes_paid, 
                                   medical_insurance, total_medical_insurance_paid,
                                   retirement_insurance, total_retirement_insurance_paid)

    __tax_percent = 0.06
    __register_date = date(2020, 1, 1)
    __patent_start = date(2020, 8, 5)
    __use_patent = True
    __patent_potential_income = 100000.0


In [82]:
# ################################### Тесты ###################################
# На тестовых данных проверим работу движка калькулятора, в данном случае я
# расчитаю размер еалогов и выплат исходя из первых принципов, будем считать,
# что данные уже загружены в памяти
# ##############################################################################
import unittest


class TestIncomeTaxCalculator(unittest.TestCase):

    def test_selectMoneyMoveByPeriods(self):
        """
            Тестируем определение движений средств, связанных с определенным периодом
            Тестовый сценарий:
                1. Формируем единый список движений средств за 2 года
                2. Для периодов 1-4 проводим выборку платежей
                3. Проверяем результат сравнивая 2 списка (отобранный с ожидаемым)
        """
        calculator = IncomeTaxCalculator(0.06, date(2020, 1, 14), True, date(2020, 3, 1), 105000)
        bank_account_money_moves = [
            Income(date(2020, 1, 10), 200000, 0),
            Income(date(2020, 2, 11), 200000, 0),
            Income(date(2020, 3, 17), 200000, 0),
            Expenditure(date(2020, 3, 1), 15900, 1, None),
            Expenditure(date(2020, 3, 19), 10000, 2, 1), # ФНС
            Expenditure(date(2020, 3, 19), 2000, 3, 1), # ФОМС
            Expenditure(date(2020, 3, 20), 10500, 4, 1), # ПФР
            Expenditure(date(2020, 4, 2), 1000, 4, 1), # ПФР
            Income(date(2020, 4, 10), 200000, 0),
            Income(date(2020, 4, 15), 115000, 0),
            Expenditure(date(2020, 4, 22), 15900, 1, None),
            Income(date(2020, 5, 10), 200000, 0),
            Income(date(2020, 6, 10), 200000, 0),
            Expenditure(date(2020, 6, 19), 20000, 2, 2),
            Expenditure(date(2020, 6, 20), 4000, 3, 2),
            Expenditure(date(2020, 6, 21), 10000, 4, 2),
            Income(date(2020, 7, 10), 200000, 0),
            Income(date(2020, 8, 10), 200000, 0),
            Income(date(2020, 9, 10), 200000, 0),
            Income(date(2020, 10, 10), 200000, 0),
            Income(date(2020, 11, 10), 200000, 0),
            Income(date(2020, 12, 10), 200000, 0)
        ]
        
        actual_period_1_money_moves = calculator.select(bank_account_money_moves, 1, 2020)
        expected_period_1_money_moves = [
            Income(date(2020, 1, 10), 200000, 0),
            Income(date(2020, 2, 11), 200000, 0),
            Income(date(2020, 3, 17), 200000, 0),
            Expenditure(date(2020, 3, 1), 15900, 1, None),
            Expenditure(date(2020, 3, 19), 10000, 2, 1), # ФНС
            Expenditure(date(2020, 3, 19), 2000, 3, 1), # ФОМС
            Expenditure(date(2020, 3, 20), 10500, 4, 1), # ПФР
            Expenditure(date(2020, 4, 2), 1000, 4, 1), # ПФР                         
        ]

        self._check_selected_money_move(expected_period_1_money_moves, actual_period_1_money_moves)

        actual_period_2_money_moves = calculator.select(bank_account_money_moves, 2, 2020)
        expected_period_2_money_moves = [
            Income(date(2020, 4, 10), 200000, 0),
            Income(date(2020, 4, 15), 115000, 0),
            Expenditure(date(2020, 4, 22), 15900, 1, None),
            Income(date(2020, 5, 10), 200000, 0),
            Income(date(2020, 6, 10), 200000, 0),
            Expenditure(date(2020, 6, 19), 20000, 2, 2),
            Expenditure(date(2020, 6, 20), 4000, 3, 2),
            Expenditure(date(2020, 6, 21), 10000, 4, 2),
        ]
        self._check_selected_money_move(expected_period_2_money_moves, actual_period_2_money_moves)

    #TODO: Реализовать тест для проверки неполного периода по страховым взносам
    def test_partialInsuranceFeesOnEmptyMoneyMoves(self):
        calculator = IncomeTaxCalculator(0.06, date(2020, 6, 15), True, date(2020, 8, 1), 105000)
        bank_account_money_moves = []
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 0, 4564.08, 17576, 0)

    def test_calculateWithPatentNoLimitOvercome(self):
        """
            Тестовый сценарий:
               1. Калькулятор настраиваем т.о., чтобы потенциальный доход был
                  меньше превышения (при котором требуется платить 1% от
                  превышения)
               2. Первые поступления на Р/С были до получения патента, поэтому
                  они считаются как 6%, а остальные не учитываются, при этом
                  эти доходы не приводят к превышению лимита
        """
        calculator = IncomeTaxCalculator(0.06, date(2019, 1, 1), True, date(2020, 6, 1), 10000)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 60000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1, None),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1, None),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1, None),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 15600, 8426, 32448, 0)

    def test_calculateWithPatentAndLimitOvercome(self):
        """
            Тестовый сценарий:
                Формируем доход т.о., чтобы он составлял 400000 до патента + 
                сумма патента 100000, которые и будут учитываться при расчете 1% в ПФР
                
        """
        calculator = IncomeTaxCalculator(0.06, date(2019, 1, 1), True, date(2020, 6, 1), 100000)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 200000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1, None),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1, None),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1, None),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 24000, 8426, 32448, 2000)

    def test_calculateNoPatentAtAll(self):
        """
            Тестовый сценарий:
                Моделируем ситуацию, когда необходимо посчитать сколько мы заплатим
                этим наглым и жадным госорганом вообще без патента (глупая затея)
        """
        calculator = IncomeTaxCalculator(0.06, date(2019, 1, 1), False, None, None)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 200000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1, None),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1, None),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1, None),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 108000, 8426, 32448, 15000)

    def _run_test_and_check(self, taxes_calculator: Calculator, 
                            bank_account_money_moves: typing.List[MoneyMove], 
                            year: int, taxes_fees: float, 
                            medical_insurance_fees: float, 
                            retirement_insurance_fees: float,
                            retirement_insuarance_extra_fees: float):
        stats = taxes_calculator.calculate(bank_account_money_moves, year)
        
        self.assertEqual(medical_insurance_fees, round(stats.get_year_stats().get_medical_insurance_fees(), 2))
        self.assertEqual(retirement_insurance_fees, round(stats.get_year_stats().get_retirement_insurance_fees(), 2))
        self.assertEqual(retirement_insuarance_extra_fees, round(stats.get_year_stats().get_extra_retirement_insurance_fees(), 2))
        self.assertEqual(taxes_fees, round(stats.get_year_stats().get_taxes(), 2))

    def _check_selected_money_move(self, expected: typing.List[MoneyMove], 
                                   actual: typing.List[MoneyMove]):
        #for a in actual:
        #    print("item with date: " + a.get_move_date().strftime('%Y-%m-%d'))
        self.assertEqual(len(expected), len(actual))
        for a in actual:
            e = list(filter(lambda x: x.get_move_date() == a.get_move_date() and
                                   x.get_movement_type() == a.get_movement_type() and
                                   x.get_value() == a.get_value(), actual))
            self.assertEqual(1, len(e))

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


In [ ]:
# ################################### Расчет ###################################
# Для расчета будем использовать следующий подход:
# Все данные будем хранить в виде tsv - файла / Excel документа на Google диске
# Необходимо определить формат хранения данных:
# Дата Сумма Назначение Комментарий
# Назначение платежа: 0 - доход, 1 - расход, 2 - уплата налогов, 
# 3 - уплата мед. страхования, 4 - уплата пенсионных взносов
# ##############################################################################

class IncomeBookCsvLoader:
    def load(self, file: str) -> typing.List[MoneyMove]:
        money_move_list = []
        lines = []
        with open(file) as f:
            lines = f.readlines()[1:]
        for line in lines:
            items = line.split('\t')
            money_move_date = datetime.strptime(items[0], "%d.%m.%Y").date()
            # TODO: Использовать для парсинга установку локалей: 
            # https://stackoverflow.com/questions/6633523/how-can-i-convert-a-string-with-dot-and-comma-into-a-float-in-python
            money_move_value = float(items[1].replace(',', '.'))
            money_move_code = items[2]
            money_move = None
            if money_move_value > 0:
                money_move = Income(money_move_date, money_move_value, money_move_code)
            else:
                #todo: имплементировать установку периода!
                money_move = Expenditure(money_move_date, money_move_value, money_move_code, items[3])
            money_move_list.append(money_move)
        return money_move_list

In [ ]:
# ######################## Расчет собственных налогов ##########################
# Берем файл с гугл драйва и для него запускаем расчет налогов и взносов
#
# ##############################################################################
import os
from google.colab import drive

# Проверка монтирования диска

drive_dev = '/content/drive'
drive_mount_point = '{0}/MyDrive'.format(drive_dev)

if not os.path.exists(drive_mount_point):
    drive.mount('/content/drive')

book_incomes_expenditures = '{0}/Документы/ИП/Book_Incomes_Expenditures_2020.tsv'.format(drive_mount_point)

# !ls '/content/drive/MyDrive/Документы/ИП'

loader = IncomeBookCsvLoader()
bank_account_money_moves = loader.load(book_incomes_expenditures)
taxes_calculator = IncomeTaxCalculator(0.06, date(2020, 6, 4), True, date(2020, 8, 1), 107460)
taxes = taxes_calculator.calculate(bank_account_money_moves, 2020)

print('Налоги в ФНС: {0}'.format(taxes.get_taxes()))
print('Отчисления в ФОМС: {0}'.format(taxes.get_medical_insurance_fees()))
print('Отчисления в ФОПС: {0}'.format(taxes.get_retirement_insurance_fees()))
print('Дополнительные отчисления в ФОПС: {0}'.format(taxes.get_extra_retirement_insurance_fees()))

Налоги в ФНС: 12821.97
Отчисления в ФОМС: 4821.544444444445
Отчисления в ФОПС: 18567.46666666667
Дополнительные отчисления в ФОПС: 211.595


In [ ]:
# ############################# Оптимизация налогов ############################
# https://vc.ru/finance/123510-instrukciya-dlya-ip-na-usn-platezhi-otchetnost
# Т.е. взносами можно гасить платежи по налогом, но они должны быть выполнены
# периодом ранее, чем уплата налогов
# ##############################################################################